# OptiGuide Example




## OptiGuide for Supply Chain Optimization

OptiGuide is a system that excels at writing code to interpret optimization solutions and answer users' questions, such as exploring the implications of changing a supply chain decision or understanding why the optimizer made a particular choice, leveraging LLMs


## New Implementation with AutoGen



Advantages of this multi-agent design with autogen:
- Collaborative Problem Solving: The collaboration among the user proxy agent and the assistant agents fosters a cooperative problem-solving environment. The agents can share information and knowledge, allowing them to complement each other's abilities and collectively arrive at better solutions. On the other hand, the Safeguard acts as a virtual adversarial checker, which can perform another safety check pass on the generated code.

- Modularity: The division of tasks into separate agents promotes modularity in the system. Each agent can be developed, tested, and maintained independently, simplifying the overall development process and facilitating code management.

- Memory Management: The OptiGuide agent's role in maintaining memory related to user interactions is crucial. The memory retention allows the agents to have context about a user's prior questions, making the decision-making process more informed and context-aware.



In [31]:
# test Gurobi installation
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import re
import requests  # for loading the example source code
import openai
import os

import autogen
from autogen.agentchat import Agent, UserProxyAgent
from optiguide import OptiGuideAgent

In [32]:
autogen.oai.ChatCompletion.start_logging()

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-4",
            "gpt4",
            "gpt-4-32k",
            "gpt-4-32k-0314",
            "gpt-3.5-turbo",
            "gpt-3.5-turbo-16k",
            "gpt-3.5-turbo-0301",
            "chatgpt-35-turbo-0301",
            "gpt-35-turbo-v0301",
        }
    }
)


Now, let's import the source code (loading from URL) and also some training examples (defined as string blow).

In [33]:
# Get the source code of the coffee example from OptiGuide's official repo
code_url = "https://raw.githubusercontent.com/microsoft/OptiGuide/main/benchmark/application/coffee.py"
response  = requests.get(code_url)
# Check if the request was successful
if response.status_code == 200:
    # Get the text content from the response
    code = response.text
else:
    raise RuntimeError("Failed to retrieve the file.")
# code = open(code_url, "r").read() # for local files


# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

import time
from gurobipy import GRB, Model

# Example data

capacity_in_supplier = {'supplier1': 150, 'supplier2': 50, 'supplier3': 100}

shipping_cost_from_supplier_to_roastery = {
    ('supplier1', 'roastery1'): 5,
    ('supplier1', 'roastery2'): 4,
.
.
.

# Solve
m.update()
model.optimize()

print(time.ctime())
if m.status == GRB.OPTIMAL:
    print(f'Optimal cost: {m.objVal}')
else:
    print("Not solved to optimality. Optimization status:", m.status)



In [34]:
# In-context learning examples.
example_qa = """
----------
Question: Why is it not recommended to use just one supplier for roastery 2?
Answer Code:
```python
z = m.addVars(suppliers, vtype=GRB.BINARY, name="z")
m.addConstr(sum(z[s] for s in suppliers) <= 1, "_")
for s in suppliers:
    m.addConstr(x[s,'roastery2'] <= capacity_in_supplier[s] * z[s], "_")
```

----------
Question: What if there's a 13% jump in the demand for light coffee at cafe1?
Answer Code:
```python
light_coffee_needed_for_cafe["cafe1"] = light_coffee_needed_for_cafe["cafe1"] * (1 + 13/100)
```

"""

Now, let's create an OptiGuide agent and also a user.

For the OptiGuide agent, we only allow "debug_times" to be 1, which means it can debug its answer once if it encountered errors.

In [35]:
%%capture
agent = OptiGuideAgent(name="OptiGuide Coffee Example",
                  source_code=code,
                   debug_times=1,
                  example_qa="",
                llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    })

user = UserProxyAgent("user", max_consecutive_auto_reply=0,
                         human_input_mode="NEVER", code_execution_config=False)


Now, let's create a user's question.

In [36]:
user.initiate_chat(agent, message="What if we prohibit shipping from supplier 1 to roastery 2?")


user (to OptiGuide Coffee Example):

What if we prohibit shipping from supplier 1 to roastery 2?

--------------------------------------------------------------------------------
OptiGuide Coffee Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Coffee Example):

```python
# Add constraint to prohibit shipping from supplier 1 to roastery 2
model.addConstr(x['supplier1', 'roastery2'] == 0, "no_shipping_supplier1_to_roastery2")
```

--------------------------------------------------------------------------------
OptiGuide Coffee Example (to safeguard):


--- Code ---
# Add constraint to prohibit shipping from supplier 1 to roastery 2
model.addConstr(x['supplier1', 'roastery2'] == 0, "no_shipping_supplier1_to_roastery2")

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to OptiGuide Coffee Example):

SAFE

----------

In [37]:
user.initiate_chat(agent, message="What is the impact of supplier1 being able to supply only half the quantity at present?")


user (to OptiGuide Coffee Example):

What is the impact of supplier1 being able to supply only half the quantity at present?

--------------------------------------------------------------------------------
OptiGuide Coffee Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Coffee Example):

```python
# Update the capacity of supplier1 to half the original value
new_capacity = capacity_in_supplier['supplier1'] / 2
capacity_in_supplier.update({'supplier1': new_capacity})

# Update the supply constraint for supplier1
model.remove(model.getConstrByName('supply_supplier1'))
model.addConstr(
    sum(x[i] for i in shipping_cost_from_supplier_to_roastery.keys()
        if i[0] == 'supplier1') <= capacity_in_supplier['supplier1'], "supply_supplier1")

# Solve the updated model
model.optimize()

# Print the new optimal cost
if model.status == GRB.OPTIMAL:
    updated_cost = model.objVal
else:
    updated_co